In [ ]:
%%capture
!pip install speechbrain
!pip install transformers

In [ ]:
import os

import torchaudio
from speechbrain.inference.VAD import VAD
from speechbrain.dataio.dataio import read_audio
from speechbrain.inference.separation import SepformerSeparation as separator

from IPython.display import Audio

import pandas as pd
from tqdm import tqdm

In [ ]:
VAD = VAD.from_hparams(source="speechbrain/vad-crdnn-libriparty", savedir='./.cache/vad-crdnn-libriparty', run_opts={"device":"cuda"})
model = separator.from_hparams(source="speechbrain/sepformer-libri2mix", savedir='./.cache/sepformer-libri2mix', run_opts={"device":"cuda"})

In [ ]:
est_sources = model.separate_file(path='speechbrain/sepformer-wsj02mix/test_mixture.wav')
est_sources.shape

In [ ]:
Audio("./audio_cache/test_mixture.wav", rate=8000)

In [ ]:
signal = read_audio("./audio_cache/test_mixture.wav").squeeze()
Audio(signal, rate=8000)

In [ ]:
est_sources.shape

In [ ]:
Audio(est_sources[:, :, 0].detach().cpu().squeeze(), rate=8000)

In [ ]:
Audio(est_sources[:, :, 1].detach().cpu().squeeze(), rate=8000)

In [ ]:
est_sources[:, :, 0]

In [ ]:
df = pd.read_csv("./data/test.csv")

In [ ]:
df

In [ ]:
df['id']

In [ ]:
if not os.path.isdir("./data/test_splitted"):
    os.mkdir("./data/test_splitted")

In [ ]:
labels = dict(id=[], path0=[], path1=[], label0=[], label1=[])

for _id in zip(tqdm(df['id'])):
    est_sources = model.separate_file(path=f"./data/test/{_id}.ogg")
    
    if not os.path.isdir(f"./data/test_splitted/{_id}/"):
        os.mkdir(f"./data/test_splitted/{_id}/")
    
    labels['id'].append(_id)
    sample_rate = 8000
    
    file0_path = f"./data/test_splitted/{_id}/0.wav"
    labels['path0'] = file0_path
    torchaudio.save(file0_path, est_sources[:, :, 0].detach().cpu(), sample_rate)
    boundaries = VAD.get_speech_segments(file0_path)
    print(boundaries)
    labels['label0'] = "speech" if 'SPEECH' in boundaries else "noise"

    file1_path = f"./data/test_splitted/{_id}/1.wav"
    labels['path1'] = file1_path
    torchaudio.save(file1_path, est_sources[:, :, 1].detach().cpu(), sample_rate)
    boundaries = VAD.get_speech_segments(file1_path)
    labels['label1'] = "speech" if 'SPEECH' in boundaries else "noise"

pd.DataFrame(labels).to_csv(f"./data/test_splitted.csv", index=False)